# Basic text mining example

Quick demonstration of a text mining example on the Consumer Financial complaints data published on the Consumer Financial Protection Bureau (CFPB) website.

Utilizes the text attribute - Consumer complaint narrative to answer one simple question: what are the top 10 key complaint word that the top 3 credit bureau agencies (Equifax, Experian, and TransUnion) received ?

Based on https://github.com/lilly-chen/Bite-sized-Machine-Learning/blob/master/Text%20Mining%20Example%201/Text%20Mining%20Example.ipynb
    
The only change is an optimisation of the fuzzyness : instead of computing it on every word of every document, I compute it in only on the word in the resulting count table.
It's a huge improovement in computing time.

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def pv(variable):
    if(len(variable)>1): print(variable, ":", eval(variable))

In [2]:
#Load library
import pandas as pd
import numpy as np
import re
from collections import Counter

In [3]:
# For dispaly purpose
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [4]:
df = pd.read_csv('/media/data-nvme/dev/datasets/ConsumerFinance/Consumer_Complaints.csv')

/media/data-nvme/dev/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Delete every line without a complain
df.dropna(subset=['Consumer complaint narrative'], inplace=True)
#df.head(3)
print('Complain data set shape: ', df.shape)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
32652,03/19/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,XXXX and Transunion are reporting incorrectly...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",LA,701XX,NaN,Consent provided,Web,03/19/2019,Closed with explanation,Yes,NaN,3184195
32668,03/19/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,XXXX and Transunion are reporting incorrectly ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",MO,NaN,NaN,Consent provided,Web,03/19/2019,Closed with explanation,Yes,NaN,3184692
33142,03/18/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,"XXXX, XXXX, and Experian need to remove the co...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,KS,660XX,NaN,Consent provided,Web,03/18/2019,Closed with non-monetary relief,Yes,NaN,3183613


Complain data set shape:  (383322, 18)


In [6]:
df['Consumer complaint narrative'][32652]

'XXXX  and Transunion are reporting incorrectly that I am 120 days past due on loans with the XXXX  - partial account numbers XXXX ; XXXX ; XXXX ; XXXX.. These accounts reflect a {$0.00} balance and a {$0.00} past due. I have contacted the two bureaus and requested these coding errors be corrected. This incorrect reporting is harming my credit score. It is  impossible to be past due on accounts that have a {$0.00} balance and {$0.00} past due.'

# Step 1: count the word frequency

For each company we compute the number of occurence of each word.

In [7]:
# Subset the data by company
EQU = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='EQUIFAX, INC.')]
EXP = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='Experian Information Solutions Inc.')]
TRU = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='TRANSUNION INTERMEDIATE HOLDINGS, INC.')]
TOTAL = df[(df['Consumer complaint narrative'].notnull())]

# Take a look how many complaint related to each company
print(len(EQU),'complaints related to Equifax')
print(len(EXP),'complaints related to Experian')
print(len(TRU),'complaints related to TransUnion')
print(len(TOTAL),'complaints in Total')

38193 complaints related to Equifax
31352 complaints related to Experian
30337 complaints related to TransUnion
383322 complaints in Total


In [8]:
# Create a empty Counter Object for the next step counting
EQU_counts = Counter()
EXP_counts = Counter()
TRU_counts = Counter()
TOTAL_counts = Counter()

EQU_lt = EQU['Consumer complaint narrative'].tolist()
EXP_lt = EXP['Consumer complaint narrative'].tolist()
TRU_lt = TRU['Consumer complaint narrative'].tolist()
TOTAL_lt = TOTAL['Consumer complaint narrative'].tolist()

In [9]:
#loop over all the words in the complaints and add up the counts
def count_word(complaints,word_counts):
    for i in range(len(complaints)):
        for word in re.split(r'\W+',  complaints[i]):
            word_counts[word] +=1

In [10]:
# count the word for each company's complaint lists
count_word(EQU_lt,EQU_counts)
count_word(EXP_lt,EXP_counts)
count_word(TRU_lt,TRU_counts)
count_word(TOTAL_lt,TOTAL_counts)

In [11]:
# extract the most common 10 words used in each company's complaint
EQU_counts_10 = EQU_counts.most_common(10)
EXP_counts_10 = EXP_counts.most_common(10)
TRU_counts_10 = TRU_counts.most_common(10)
TOTAL_counts_10 = TOTAL_counts.most_common(10)


In [12]:
# convert to dataframe for display
EQU_df = pd.DataFrame({'most 10 common (EQU)':EQU_counts_10})
EXP_df = pd.DataFrame({'most 10 common (EXP)':EXP_counts_10})
TRU_df = pd.DataFrame({'most  10 common (TRU)':TRU_counts_10})
Total_df = pd.DataFrame({'most 10 common (Total)':TOTAL_counts_10})

display('EQU_df', 'EXP_df', 'TRU_df', 'Total_df')

,most 10 common (EQU)
0,"(XXXX, 404363)"
1,"(the, 187644)"
2,"(I, 165919)"
3,"(to, 165780)"
4,"(and, 140836)"
5,"(my, 114009)"
6,"(a, 93195)"
7,"(of, 91038)"
8,"(XX, 86436)"
9,"(that, 80382)"


As you can see, it's useless, we get only common english word.

We will divide the count for a word for a company by the total number of occurence of this word in all company.



# Step2: Calculate the frequent ratio

In [13]:
# Again, create Counter object for ratio calculation
EQU_ratios = Counter()
EXP_ratios = Counter()
TRU_ratios = Counter()

In [14]:
def calculate_ratio(word_counts,ratios):
    for word in list(word_counts):
        ratio = word_counts[word] / float(TOTAL_counts[word]+1) # +1 to avoid dividing by zero
        ratios[word] = ratio

In [15]:
# calculate the ratio for each company's complaint words
calculate_ratio(EQU_counts,EQU_ratios)
calculate_ratio(EXP_counts,EXP_ratios)
calculate_ratio(TRU_counts,TRU_ratios)

In [16]:
# words with the highest ratio 
EQU_df = pd.DataFrame({'most_common (EQU)':EQU_ratios.most_common(10)})
EXP_df = pd.DataFrame({'most_common (EXP)':EXP_ratios.most_common(10)})
TRU_df = pd.DataFrame({'most_common (TRU)':TRU_ratios.most_common(10)})

display('EQU_df', 'EXP_df', 'TRU_df')

,most_common (EQU)
0,"(equifaxsecurity2017, 0.9981203007518797)"
1,"(Equifaxs, 0.9924242424242424)"
2,"(Equifax, 0.9903262632938773)"
3,"(TrustedID, 0.9901960784313726)"
4,"(EQUIFAX, 0.9873164218958611)"
5,"(Eqifax, 0.98)"
6,"(EquiFax, 0.979381443298969)"
7,"(Equfax, 0.9791666666666666)"
8,"(Exquifax, 0.9791666666666666)"
9,"(Intruders, 0.9761904761904762)"


Seems better, but we saw only variation of the company name.
We have to group them.

# Step 3: improve the ratio

We will use fuzzyness algorithm to exclude mispelled company name. Basically, it calculate a distance (called Levenshtein distance) to measure the difference between two words.

The drawback is that it takes a very long time, we have to be careful not to run in on all words of the dataset.

In [17]:
#!pip install fuzzywuzzy

In [18]:
# illustrate how the fuzzywuzzy.process work
# 
from fuzzywuzzy import fuzz

misspelled1 = 'Experion'.lower()
misspelled2 = 'EXPERIANS'.lower()
misspelled3 = 'Experia'.lower()
match = 'Experian'.lower()
fuzzy_score1 = fuzz.ratio(misspelled1, match)
fuzzy_score2 = fuzz.ratio(misspelled2, match)
fuzzy_score3 = fuzz.ratio(misspelled3, match)
print(misspelled1, fuzzy_score1)
print(misspelled2, fuzzy_score2)
print(misspelled3, fuzzy_score3)

experion 88
experians 94
experia 93


/media/data-nvme/dev/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:
from fuzzywuzzy import fuzz

def remove_duplicate_fuzzy(counter, master_word):
    for word in counter:
        if word == master_word:
            continue
        fuzzy_score = fuzz.ratio(word.lower(), master_word.lower())
        if word == 'Experion':
            print(word, fuzzy_score)
        if fuzzy_score >= 85:
            #print(word, fuzzy_score)
            counter[master_word] = counter[master_word] + counter[word]
            # Correct also the TOTAL Count
            TOTAL_counts[master_word] = TOTAL_counts[master_word] + counter[word]
            TOTAL_counts[word] = TOTAL_counts[word] - counter[word]
            #print(counter[word], counter[master_word])
            counter[word] = 0

EQU_counts_fuzzy = EQU_counts.copy()
EXP_counts_fuzzy = EXP_counts.copy()
TRU_counts_fuzzy = TRU_counts.copy()
            
remove_duplicate_fuzzy(EQU_counts_fuzzy, "Equifax")
remove_duplicate_fuzzy(EXP_counts_fuzzy, "Experian")
remove_duplicate_fuzzy(TRU_counts_fuzzy, "TransUnion")
#EQU_counts_new.most_common(10)

Experion 88


In [20]:
# Create a empty Counter Object for the next step counting
EQU_counts2 = Counter()
EXP_counts2 = Counter()
TRU_counts2 = Counter()

# Again, create Counter object for ratio calculation
EQU_ratios2 = Counter()
EXP_ratios2 = Counter()
TRU_ratios2 = Counter()

In [21]:
calculate_ratio(EQU_counts_fuzzy,EQU_ratios2)
calculate_ratio(EXP_counts_fuzzy,EXP_ratios2)
calculate_ratio(TRU_counts_fuzzy,TRU_ratios2)

EQU_df = pd.DataFrame({'most_common (EQU)':EQU_ratios2.most_common(10)})
EXP_df = pd.DataFrame({'most_common (EXP)':EXP_ratios2.most_common(10)})
TRU_df = pd.DataFrame({'most_common (TRU)':TRU_ratios2.most_common(10)})

display('EQU_df', 'EXP_df', 'TRU_df')

,most_common (EQU)
0,"(equifaxsecurity2017, 0.9981203007518797)"
1,"(Equifax, 0.99153685759843)"
2,"(TrustedID, 0.9901960784313726)"
3,"(Intruders, 0.9761904761904762)"
4,"(XXXXEquifax, 0.9736842105263158)"
5,"(segmentation, 0.9591836734693877)"
6,"(trustedid, 0.9565217391304348)"
7,"(realleges, 0.9565217391304348)"
8,"(EquifaxXXXX, 0.95)"
9,"(trustedidpremier, 0.9473684210526315)"


That's better, we saw that Equifax clients suffer from TrustedID and Intruders.

Experian clients have Geographical problems.

TransUnion clients are concerned about 3rdpartyinfo.

# Future improovement

- This was a little introduction to text mining, but we could saw many limitation : we use only words and not their context : maybe some clients tells that TrustedID is great and complain about other things.

- We do not take care of misspelling other than for company name. We need to have a more generic approach, like lemmatization or stemming.

- Use tokenization : put all word in lowercase, remove stop words, remove XXXX...

- Use TF-IDF(https://towardsdatascience.com/tfidf-for-piece-of-text-in-python-43feccaa74f8) to give priority to word that is not in all documents.

- Use word embeddings